In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


import pandas as pd
from pandas_datareader import data

import numpy as np, numpy.random
from numpy import mean

import random

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

from datetime import datetime, timedelta

from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy import stats

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

import pmdarima
import arch

from IPython.display import display, HTML

import time
from tqdm import tqdm
from scipy.stats import zscore
import os



In [2]:
def extract_prices(start_date,end_date,symbols,portfolioWeights,portfolioValue,backtestduration=0):
    dim=len(symbols)
    for symbol in symbols:
        dfprices = data.DataReader(symbols, start=start_date, end=end_date, data_source='yahoo')
        dfprices = dfprices[['Adj Close']]
    dfprices.columns=[' '.join(col).strip() for col in dfprices.columns.values]

    priceAtEndDate=[]
    for symbol in symbols:
        priceAtEndDate.append(dfprices[[f'Adj Close {symbol}']][-(backtestduration+1):].values[0][0])

    noOfShares=[]
    portfolioValPerSymbol=[x * portfolioValue for x in portfolioWeights]
    for i in range(0,len(symbols)):
        noOfShares.append(portfolioValPerSymbol[i]/priceAtEndDate[i])
    noOfShares=[round(element, 5) for element in noOfShares]
    listOfColumns=dfprices.columns.tolist()
    dfprices["Adj Close Portfolio"]=dfprices[listOfColumns].mul(noOfShares).sum(1)

    share_split_table=dfprices.tail(1).T
    share_split_table=share_split_table.iloc[:-1]
    share_split_table["Share"]=symbols
    share_split_table["No Of Shares"]=noOfShares
    share_split_table.columns=["Price At "+end_date,"Share Name","No Of Shares"]
    share_split_table["Value At "+end_date]=share_split_table["No Of Shares"]*share_split_table["Price At "+end_date]
    share_split_table.index=share_split_table["Share Name"]
    share_split_table=share_split_table[["Share Name","Price At "+end_date,"No Of Shares","Value At "+end_date]]
    share_split_table=share_split_table.round(3)
    share_split_table=share_split_table.append(share_split_table.sum(numeric_only=True), ignore_index=True)
    share_split_table.at[len(symbols),'No Of Shares']=np.nan
    share_split_table.at[len(symbols),'Price At '+end_date]=np.nan
    share_split_table.at[len(symbols),'Share Name']="Portfolio"
    share_split_table["Weights"]=portfolioWeights+["1"]
    share_split_table = share_split_table[['Share Name', 'Weights', 'Price At '+end_date, 'No Of Shares', "Value At "+end_date]]

    print(f"Extracted {len(dfprices)} days worth of data for {len(symbols)} counters with {dfprices.isnull().sum().sum()} missing data")

    return dfprices, noOfShares, share_split_table


In [3]:
start_date = '2016-12-30'
end_date='2019-06-11'

symbols=['FB','NVDA','AMZN','NFLX','GOOG']
portfolioWeights=[0.3,0.1,0.2,0.2,0.2]
portfolioValue=5000

dfprices, noOfShares, share_split_table = extract_prices(start_date,end_date,symbols,portfolioWeights,portfolioValue)


Extracted 614 days worth of data for 5 counters with 0 missing data


In [4]:
dfprices

,Adj Close FB,Adj Close NVDA,Adj Close AMZN,Adj Close NFLX,Adj Close GOOG,Adj Close Portfolio
Date,,,,,,
2016-12-30,115.050003,105.443413,749.869995,123.800003,771.820007,2790.175515
2017-01-03,116.860001,100.770882,753.669983,127.489998,786.140015,2815.688294
2017-01-04,118.690002,103.121964,757.179993,129.410004,786.900024,2846.979122
2017-01-05,120.669998,100.504166,780.450012,131.809998,794.020020,2880.861848
2017-01-06,123.410004,101.847641,795.989990,131.070007,806.150024,2925.886281
...,...,...,...,...,...,...
2019-06-05,168.169998,140.803894,1738.500000,355.730011,1042.219971,4796.638994
2019-06-06,168.330002,143.295471,1754.359985,357.130005,1044.339966,4820.739262
2019-06-07,173.350006,145.009659,1804.030029,360.869995,1066.040039,4926.138557


In [5]:
noOfShares

[8.42223, 3.32797, 0.53657, 2.84681, 0.92702]

In [7]:
share_split_table

,Share Name,Weights,Price At 2019-06-11,No Of Shares,Value At 2019-06-11
0,FB,0.3,178.100,8.422,1499.999
1,NVDA,0.1,150.242,3.328,500.001
2,AMZN,0.2,1863.700,0.537,1000.005
3,NFLX,0.2,351.270,2.847,999.999
4,GOOG,0.2,1078.720,0.927,999.995
5,Portfolio,1,NaN,NaN,4999.999


In [8]:
dfprices = data.DataReader(symbols, start=start_date, end=end_date, data_source='yahoo')


In [10]:
dfprices[["Adj Close"]]

Attributes   Adj Close                                                  
Symbols             FB        NVDA         AMZN        NFLX         GOOG
Date                                                                    
2016-12-30  115.050003  105.443413   749.869995  123.800003   771.820007
2017-01-03  116.860001  100.770882   753.669983  127.489998   786.140015
2017-01-04  118.690002  103.121964   757.179993  129.410004   786.900024
2017-01-05  120.669998  100.504166   780.450012  131.809998   794.020020
2017-01-06  123.410004  101.847641   795.989990  131.070007   806.150024
...                ...         ...          ...         ...          ...
2019-06-05  168.169998  140.803894  1738.500000  355.730011  1042.219971
2019-06-06  168.330002  143.295471  1754.359985  357.130005  1044.339966
2019-06-07  173.350006  145.009659  1804.030029  360.869995  1066.040039
2019-06-10  174.820007  147.929779  1860.630005  352.010010  1080.380005
2019-06-11  178.100006  150.241959  1863.699951  351.269989  1078.719971

[614 rows x 5 columns]